# Code Used to analyse the convergence properties of RPM on Factor Analysis

## Helpers

In [1]:
# Imports
import copy
import pickle
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from utils_process import plot_loss
from matplotlib.pyplot import get_cmap

from rpfa import RPM
from data import RPMData

# GPUs ?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data type: float64 / float32
data_type = torch.float32
torch.set_default_dtype(data_type)

def generate_mixture_factor_analysis(
    num_observations, 
    dim_observations,
    num_factors,
    dim_latent_true = 1,
    cluster_num = 1,
    cluster_scattering = 10,
    amplitude_cluster = 8,  
    variance_scales = [1],
):
    
    centroids = cluster_scattering * _init_centroids(
        cluster_num,
        dim_latent_true,
        ite_max=10000,
        optimizer=lambda x: torch.optim.Adam(x, lr=1e-2),
    )[0]

    # Sample cluster
    centroids_sample = centroids[np.random.choice(range(cluster_num), num_observations, replace=True)]
    
    # Add some gaussian noise around cluster mean
    zlatent = torch.randn(num_observations, dim_latent_true, device=device) +  centroids_sample
    
    # E(x) = Az
    link_factors = torch.rand(num_factors, dim_observations, dim_latent_true, device=device)
    
    # Variances
    vari_factors = 0.1 * torch.eye(dim_observations, device=device).unsqueeze(0).repeat(num_factors, 1, 1)
    
    # Scale Variances
    for JJ in range(num_factors):
        vari_factors[JJ] *= variance_scales[JJ]
    
    # Sample each Factor Noise
    noise = torch.sqrt(
        vari_factors.diagonal(dim1=-1, dim2=-2)
    ).unsqueeze(1) * torch.randn(num_factors, num_observations, dim_observations, device=device)
    
    
    # Build Observations
    observations = torch.matmul(link_factors.unsqueeze(1), zlatent.unsqueeze(0).unsqueeze(-1)).squeeze(-1) + noise
    
    # RPM Compatible Observations    
    observations = [obsi for obsi in observations]
    
    return observations, zlatent

In [ ]:
# Random seeds
torch.manual_seed(100) 
np.random.seed(100)    

# Fit params
num_factors =  2
dim_latent_true = 1
dim_observations = 10
num_observations = 500
variance_scales = [1, 1]

# Generate Observations
observations, zlatent = generate_mixture_factor_analysis(
    num_observations, 
    dim_observations,
    num_factors,
    cluster_num = 1,
    cluster_scattering = 10,
    amplitude_cluster = 8,  
    variance_scales = variance_scales,
)

# RPM Parameters
factors_params = {
    'dim_hidden': [[], []],
    'non_linearity': [torch.nn.Identity(), torch.nn.Identity()],
    'optimizer': lambda params: torch.optim.AdamW(params=params, lr=1e-2, weight_decay=0.0),
}
prior_params = {
    'num_centroids': 1,
    'optimizer': lambda params: torch.optim.AdamW(params=params, lr=1e-2, weight_decay=0.0),
}
fit_params0 = {
    'num_epoch': 3000,
    'dim_latent': 1,
    'auxiliary_mode': 'flexible', #flexible, semi_flexible, constrained_prior
    'factors_params': factors_params,
    'prior_params': prior_params,
    'pct': 1.1,
} 

# Fit modes
modes = [
    'flexible',
    'constrained_prior',
    'semi_flexible',
]

# Sample sizes
nums = list(np.floor(np.logspace(1, 3, 5)).astype(int))

# Number of random seeds
num_seeds = 10

# Loop over seeds and sample sizes
rpms = []
results = []
for mode in modes:
    for num in nums:
        for seed in range(num_seeds):

            print('Mode: ' + mode + '| N = ' + str(num) + '| seed = ' + str(seed))    
            
            observation = RPMData(
                [obsi[:num] for obsi in observations],
                zlatent[:num]
            )
            
            fit_params = copy.deepcopy(fit_params0)
            fit_params['auxiliary_mode'] = mode
            
            rpm = RPM(
                observations=observation,
                fit_params=fit_params,
            )
            
            rpm.fit()
            rpms.append(rpm)
    
            results.append(
                {
                    'mode': mode,
                    'num': num,
                    'loss': rpm.loss_tot
                }
            )

[10, 31, 100, 316, 1000]
Mode: flexible| N = 10| seed = 0
RPM on cpu
Mode: flexible| N = 10| seed = 1
RPM on cpu
Mode: flexible| N = 10| seed = 2
RPM on cpu
Mode: flexible| N = 10| seed = 3
RPM on cpu
Mode: flexible| N = 10| seed = 4
RPM on cpu
Mode: flexible| N = 10| seed = 5
RPM on cpu
Mode: flexible| N = 10| seed = 6
RPM on cpu
Mode: flexible| N = 10| seed = 7
RPM on cpu
Mode: flexible| N = 10| seed = 8
RPM on cpu
Mode: flexible| N = 10| seed = 9
RPM on cpu
Mode: flexible| N = 31| seed = 0
RPM on cpu
Mode: flexible| N = 31| seed = 1
RPM on cpu
Mode: flexible| N = 31| seed = 2
RPM on cpu
Mode: flexible| N = 31| seed = 3
RPM on cpu
Mode: flexible| N = 31| seed = 4
RPM on cpu
Mode: flexible| N = 31| seed = 5
RPM on cpu
Mode: flexible| N = 31| seed = 6
RPM on cpu
Mode: flexible| N = 31| seed = 7
RPM on cpu
Mode: flexible| N = 31| seed = 8
RPM on cpu
Mode: flexible| N = 31| seed = 9
RPM on cpu
Mode: flexible| N = 100| seed = 0
RPM on cpu
Mode: flexible| N = 100| seed = 1
RPM on cpu
Mode:

In [8]:
import pandas as pd
df = pd.DataFrame(results)
#df.to_csv(path_or_buf='flexible_study_1.cvs')